In [1]:
import sys
!{sys.executable} -m pip install -qU langchain-huggingface



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [37]:

pip install -U langchain langchain-core

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
pip install sentence-transformers langchain_ollama owlready2

In [38]:
def scenario_splitter_llm(story, llm):
    story_to_scenarios_prompt = f"""
    You are an assistant that extracts logical or factual scenarios from a given story.

    A scenario is a small, self-contained statement or short passage that expresses one or more closely related facts, events, or claims that can later be checked for correctness or consistency using an ontology.
    ---

    ### Guidelines:
    - Each scenario should capture a complete idea, including all details that are logically connected (for example, cause–effect, contrast, or relationship).
    - If two or more statements are relevant to each other (e.g., one qualifies, contradicts, or explains the other), combine them into one scenario.
    - Preserve contextual and relational details - who, what, where, when, and why.
    - Avoid redundancy - do not restate the same information.
    - Keep each scenario brief but complete (usually one to three sentences).
    - Include implicit facts when they are important (e.g., “John’s wife Amira” implies John is married to Amira).
    - Ensure that every piece of relevant information from the story appears in at least one scenario.

    ---

    Example

    Story:
    “John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the Eiffel Tower.”

    Extracted Scenarios:
    1. John is 15 years old and is married to Amira.
    2. John and Amira are on vacation in Italy.
    3. John and Amira have a daughter named Anna.
    4. Anna wants to visit the Eiffel Tower.

    ---

    Now extract the scenarios from the following story and number them clearly:

    {story}
    """

    # Query the LLM
    response = llm.invoke(story_to_scenarios_prompt)
    if hasattr(response, "content"):  # LangChain AIMessage
        output_text = response.content
    elif isinstance(response, dict) and "content" in response:
        output_text = response["content"]
    elif isinstance(response, str):
        output_text = response
    else:
        raise TypeError(f"Unexpected LLM response type: {type(response)}")

    # Extract list items using regex
    import re
    scenarios = re.findall(r'(?:\d+\.\s*)(.+)', output_text)
    scenarios = [s.strip() for s in scenarios if s.strip()]

    # If the LLM doesnt number them, fallback to line splitting
    if not scenarios:
        scenarios = [line.strip("-• \t") for line in output_text.splitlines() if line.strip()]

    return scenarios


In [39]:

from typing import List
from langchain_core.prompts import PromptTemplate

def incremental_reasoning_no_rag(scenarios, custom_prompt, llm):
    """
    Performs step-by-step reasoning on a sequence of scenarios, using only the
    accumulated story so far (no ontology/vector store retrieval).
    """
    updated_scenarios = []
    results = []
    accumulated_story = ""  # stores all accepted or corrected story so far

    for i, scenario in enumerate(scenarios, start=1):
        # 1. Construct reasoning context (story so far only)
        full_context = f"Story so far:\n{accumulated_story.strip() or '(none so far)'}\n"

        # 2. Build the complete prompt using the template
        rendered_prompt = custom_prompt.invoke({
            "question": scenario.strip(),
            "context": full_context.strip(),
        })

        # 3. Invoke the LLM
        llm_response = llm.invoke(rendered_prompt)
        response_text = llm_response.content.strip()

        # 4. Parse the model output (Consistency + Updated scenario)
        updated_scenario = scenario.strip()
        consistency = "Unknown"

        for line in response_text.splitlines():
            line_lower = line.strip().lower()
            if line_lower.startswith("updated scenario:"):
                updated_scenario = line.split(":", 1)[1].strip()
            elif line_lower.startswith("consistency:"):
                consistency = line.split(":", 1)[1].strip()

        # 6. Check consistency, update if not consistent
        #If consistent, use unchanged scenario
        if(consistency.lower().strip() == "consistent"):
            #Update accumulated story so future reasoning uses this to fix scenarios
            accumulated_story = (accumulated_story + "\n" + scenario).strip()
            #Update scenario
            updated_scenarios.append(scenario)

        #If not consistent, use llm updated scenario
        else:
            #Update accumulated story so future reasoning uses this to fix scenarios
            accumulated_story = (accumulated_story + "\n" + updated_scenario).strip()
            #Update scenario
            updated_scenarios.append(updated_scenario)

        # 7. Store reasoning results
        results.append({
            "step": i,
            "original_scenario": scenario.strip(),
            "updated_scenario": updated_scenario,
            "consistency": consistency,
            "llm_raw_output": response_text.strip(),
            "story_so_far": accumulated_story.strip(),
        })

    return updated_scenarios, results



In [40]:
scenario_fixing_prompt_no_rag = PromptTemplate.from_template("""
    You are a careful reasoning assistant that ensures each scenario in a story
    is logically consistent with the previously verified parts of the same story.

    You will be given:
    1) The story so far - previously verified or corrected facts.
    2) A new scenario - the next statement to check.

    Your task:
    - Determine whether the new scenario is **consistent** with the story so far.
    - If inconsistent, rewrite the scenario minimally to make it consistent.
    - Prefer small, natural edits that preserve meaning and story flow.

    **STRICT OUTPUT FORMAT (must follow exactly)**

    Consistency: [Consistent / Inconsistent]

    Updated scenario: [Corrected or unchanged scenario; if unchanged, copy the original. DO NOT OUTPUT "No change" or "unchanged"]

    Story so far:
    John is 15 years old.

    Next scenario:
    John is married to Amira.

    Expected output:                                                   
    Let's think step-by-step.
    Explanation:                                                   
    Consistency: Inconsistent
    Updated scenario: John is 23 years old and is married to Amira.

    Now analyze using the provided context below.
    {context}

    Next scenario:
    {question}
                                                        
    Let's think step-by-step
    """)

In [41]:
from langchain_core.prompts import PromptTemplate

fix_story_prompt = PromptTemplate.from_template("""
You are a reasoning assistant tasked with fixing inconsistencies in a story.

You are given:

The original story:
{original_story}
                                           
The original inconsistent scenarios/facts taken from the original story
{scenarios}

A list of updated, consistent scenarios/facts:
{updated_scenarios}

Your task is to rewrite the new updated scenarios into the original story. Make minimal changes to the original story; preserve the writing style, narrative flow, and character details as much as possible. 

Return only the fixed story.
""")

In [42]:
def fix_story(llm, story, scenarios, updated_scenarios):
    fixed_story = llm.invoke(fix_story_prompt.format(
    original_story=story,
    scenarios=scenarios,
    updated_scenarios="\n".join(updated_scenarios)
))
    return fixed_story.content

In [56]:
from typing import List, TypedDict, Optional, Dict, Any
from langgraph.graph import StateGraph, START, END
from langchain_core.prompts import PromptTemplate
from langchain_ollama import ChatOllama

class StoryStateNoRAG(TypedDict, total=False):
    story: str
    scenarios: List[str]
    updated_scenarios: List[str]
    results: Any
    fixed_story: str
    llm: Any
    loop_count: int #how many times check consistency node has been done

#Create scenarions based on story
def split_scenarios_node_no_rag(state: StoryStateNoRAG) -> Dict[str, Any]:
    story = state.get("story", "")
    llm = state.get("llm")
    scenarios = scenario_splitter_llm(story, llm)
    return {"scenarios": scenarios}

#Fixes the scenarios incrementally
def check_consistency_node_no_rag(state: StoryStateNoRAG) -> Dict[str, Any]:
    scenarios = state.get("updated_scenarios") or state.get("scenarios")
    llm = state.get("llm")
    loop_count = state.get("loop_count", 0)

    updated_scenarios, results = incremental_reasoning_no_rag(scenarios, scenario_fixing_prompt_no_rag, llm)
    return {"updated_scenarios" : updated_scenarios, "results": results, "loop_count": loop_count + 1}

#Fixes the story
def fix_story_node_no_rag(state: StoryStateNoRAG) -> Dict[str, Any]:
    story = state.get("story", "")
    scenarios = state.get("scenarios")
    updated_scenarios = state.get("updated_scenarios")
    llm = state.get("llm")
    fixed_story = fix_story(llm, story, scenarios, updated_scenarios)

    return { "fixed_story": fixed_story}

#condition for looping consistency check
def should_repeat_check_consistency(state: StoryStateNoRAG):
    #Limit to N loops
    if state["loop_count"] < 2:#Do total of 2 times
        return "repeat"
    return "done"

builder_no_rag = StateGraph(StoryStateNoRAG)
builder_no_rag.add_node("split_scenarios_no_rag", split_scenarios_node_no_rag)
builder_no_rag.add_node("check_consistency_no_rag", check_consistency_node_no_rag)
builder_no_rag.add_node("fix_story_no_rag", fix_story_node_no_rag)

builder_no_rag.add_edge(START, "split_scenarios_no_rag")
builder_no_rag.add_edge("split_scenarios_no_rag", "check_consistency_no_rag")
builder_no_rag.add_conditional_edges(
    "check_consistency_no_rag",
    should_repeat_check_consistency,
    {
        "repeat": "check_consistency_no_rag", #loop
        "done": "fix_story_no_rag",           #continue
    },
)
builder_no_rag.add_edge("fix_story_no_rag", END)

graph_no_rag = builder_no_rag.compile()

#-------------------------------------------------
#   Stories. 
#   Comment out all other stories you dont use.
#-------------------------------------------------

#Story 1
#story = """John is 15 years old and is on vacation with his wife Amira in Italy. Their daughter Anna can’t wait to visit the famous landmark of Florance, the Eiffel Tower. After that they will go out to eat. John suggests they eat pizzas since Italy is famous for them, they will eat at restaurant Riccolo located in Florance.
#Since Anna has a vitamin C deficiency she will order a pizza that contains tomatoes for vitamin C, so she will get the pizza bianca. John will get the classic Margherita pizza and Amira orders a pepperoni pizza.
#They sit by the window of the small restaurant the air filled with the smell of garlic and baking dough. Anna swings her legs impatiently under the table, still talking about the Eiffel Tower, while Amira flips through a guidebook about Florance.
#15 minutes later the farmer named Leo from the restaurant brings their pizzas and they eat, and Anna says to John how cool it is that the waiter is from France."""

#Story 2
story = "Jack and his family are celebrating his birthday, Jack just turned 235 years old. Jack loves talking about himself and everything he has done in his life, thats why many people would consider Jack a reserved person. This year, Jack’s family decided to visit the beautiful city of Quito, famous for its landmark La Virgen del Panecillo, which overlooks the city from a tall hill. Since the city doesn’t have mountainous terrain and is easy to walk through, Jack enjoys strolling through the streets without any trouble. It’s quite convenient, because Jack can’t walk very far due to his asthma. Luckily, Jack brought his medication for his asthma, so whenever he’s out of breath, he can simply take his antihistamine pills."

#Story 3
#story = """Emily and her son Michael decided to adopt a cat from the local shelter in Denver, a walkable city surrounded by tall mountains. The city’s most famous landmark, the Red Rocks Amphitheatre, could be seen from their apartment window. Michael was thrilled to finally have a pet to take on walks through the flat city streets. The shelter worker handed them a small brown dog named Whiskers, explaining that he meows loudly when he wants food.
#Later that afternoon, Michael went to his scheduled health check-up for obesity, which the doctor said was likely caused by living in Denver. After the appointment, Michael decided to walk Whiskers around the flat, car-free neighborhood for some exercise. Denver has a large population, which makes it one of the largest cities in the USA. Michael finds Denver the quietest city he has ever been to.
#"""
#Story 4
#story = "Jason loves to work outside on the Farm, which is why he became a surgeon. He and his family live just outside of Barcelona in France where he grows apples. He is a father of his daughter Amelie. When they visit the city Amelie gets very excited for all the vegan food, such as Chicken Tikka Masala, since she is vegetarian by choice but also lactose intolerant. Whenever they visit, they take their pet alligator Sally since she loves to meet other pets and is very social."

#Story 5
#story = """Lara traveled to Rome with her friend Amira to enjoy Italian cuisine, they were excited to try everything. However, it wasn’t easy since Lara was vegetarian, and her best friend was gluten-free. They finally both enjoyed the Pizza and Pasta. They also explored the beautiful city, walking everywhere they could. Lara had to use her EpiPen daily since she was highly allergic to animal hair and in many places they encountered street animals, such as cats and dogs.
#Lara loved the smell of baked bread from every corner, even though her friend couldn’t try any. When Lara returned home, she wanted to make food that everyone could enjoy, so she opened a small bakery and decided it would be completely gluten-free. Her favourite creation was bread made from rye.
#"""

llm = ChatOllama(
    #model="phi3:mini",
    model = "llama3.1"
)

initial_state: StoryStateNoRAG = {
    "story": story,
    "llm": llm,
    "loop_count": 0
}
from IPython.display import Image, display
final_state = graph_no_rag.invoke(initial_state)


In [57]:
#Print Created scenarios from Story
final_state["scenarios"]

['Jack is 235 years old and has a family that celebrates his birthday.',
 'Jack is considered a reserved person due to his love for talking about himself.',
 "Jack's family decided to visit the city of Quito, Ecuador this year.",
 'La Virgen del Panecillo is a famous landmark in Quito that overlooks the city from a tall hill.',
 'The city of Quito has flat terrain and is easy to walk through.',
 'Jack has asthma, which makes it difficult for him to walk far.',
 'Jack brought his medication (antihistamine pills) with him on the trip to manage his asthma symptoms.']

In [58]:
#Print Updated scenarios from Story
final_state["updated_scenarios"]

['Jack is 115 years old and has a family that celebrates his birthday.',
 '',
 "Jack's family decided to visit the city of Quito, Ecuador this year.",
 'La Virgen del Panecillo is a famous landmark in Quito that overlooks the city from a tall hill.',
 "La Virgen del Panecillo is a famous landmark in Quito that overlooks the city from a tall hill, which contrasts with Quito's generally flat terrain and easy walkability.",
 'Jack has asthma, which makes it difficult for him to walk far.',
 'Jack brought inhalers with him on the trip to manage his asthma symptoms.']

In [59]:
#Print Final fixed Story
print(final_state["fixed_story"])

Here's the rewritten story with the inconsistencies fixed:

Jack and his family are celebrating his birthday, Jack just turned 115 years old. Jack loves talking about himself and everything he has done in his life, that's why many people would consider Jack a reserved person. This year, Jack's family decided to visit the beautiful city of Quito, famous for its landmark La Virgen del Panecillo, which overlooks the city from a tall hill. Although the city doesn't have mountainous terrain and is easy to walk through, Jack has asthma, making it difficult for him to walk far. Luckily, Jack brought his inhalers with him on the trip to manage his asthma symptoms.

Despite this limitation, Jack enjoys strolling through the streets without any trouble, taking regular breaks whenever he's out of breath. It's quite convenient, thanks to Quito's generally flat terrain that allows for easy walking, and La Virgen del Panecillo can be seen from a distance on one of these walks.


In [60]:
#Print reasoning etc from fixing scenarios function (Check explantion & ontology used to figure out llm reasoning)
reasoning_data = final_state["results"]
reasoning_data

[{'step': 1,
  'original_scenario': 'Jack is 235 years old and has a family that celebrates his birthday.',
  'updated_scenario': 'Jack is 115 years old and has a family that celebrates his birthday.',
  'consistency': 'Inconsistent',
  'llm_raw_output': "Consistency: Inconsistent\n\nUpdated scenario: Jack is 115 years old and has a family that celebrates his birthday.\n\nExplanation: \nA human cannot live up to 235 years, as the maximum recorded age in history is about 122 years (Jeanne Calment). Therefore, the original statement is inconsistent with known facts. To make it consistent, I have revised Jack's age to be within the biologically possible range.",
  'story_so_far': 'Jack is 115 years old and has a family that celebrates his birthday.'},
 {'step': 2,
  'original_scenario': 'Jack is considered a reserved person due to his love for talking about himself.',
  'updated_scenario': '',
  'consistency': 'Inconsistent',
  'llm_raw_output': 'To ensure consistency, let\'s analyze the 

In [ ]:
import json

# Create a combined export dictionary
export_data = {
    "original_story": final_state.get("story", ""),
    "scenarios": final_state.get("scenarios", []),
    "updated_scenarios": final_state.get("updated_scenarios", []),
    "fixed_story": final_state.get("fixed_story", ""),
    "reasoning_results": final_state.get("results", []),
}

# Save it as JSON
with open("story+reasoning_output.json", "w", encoding="utf-8") as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

print("Saved everything to json file")


Saved everything to json file


In [55]:
all_runs = []

for i in range(10):
    print(f"running {i+1}/10")
    final_state = graph_no_rag.invoke(initial_state)

    all_runs.append({
        "scenarios": final_state.get("scenarios", []),
        "updated_scenarios": final_state.get("updated_scenarios", []),
        "fixed_story": final_state.get("fixed_story", "")
    })


export_data = {
    "original_story": story,
    "iterations": all_runs
}

with open("base_model_final_story5_10runs.json", "w", encoding="utf-8") as f:
    json.dump(export_data, f, indent=2, ensure_ascii=False)

running 1/10
running 2/10
running 3/10
running 4/10
running 5/10
running 6/10
running 7/10
running 8/10
running 9/10
running 10/10
